In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.colors as plt_col
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.dates as pltd
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings("ignore")

In [12]:
def plot_map(data):
    ax = fig.add_subplot(spec[nrow,0], projection=ccrs.Mollweide(central_longitude=240))
    ax.add_feature(cfeature.COASTLINE,zorder=11)
    ax.add_feature(cfeature.LAND,zorder=10)
    #gl = ax.gridlines(crs=proj, draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
    
    P = ax.pcolormesh(lon_crn,lat_crn,data[1:-1,1:-1],transform = ccrs.PlateCarree(), cmap=mymap)
    plt.setp(ax.spines.values(), color='black',zorder=12)
    
    #pos = ax.get_position()
    #cax = fig.add_axes([pos.x0+1.06*pos.width, pos.y0, .06*pos.width, pos.height])
    #plt.colorbar(P,cax=cax)
    ax.set_title((plt_tits[nrow]),fontsize=16, pad=10)
    
    if nrow==0:
        pos = ax.get_position()
        all_handles=[]
        for ncol in range(len(legend_titles)):
            leg_patch = mpatches.Patch(color=col_trm[ncol], label=legend_titles[ncol])
            all_handles.append(leg_patch)
        ax.legend(bbox_to_anchor=(0, -0.4, 1., .2), handles=all_handles, ncol = len(legend_titles), mode="expand", frameon=False, prop={'size': 20})
        

In [3]:
def calc_lims(tim_per):
    
    if tim_per == "his":
        arch_dir_ann = '/archive/oar.gfdl.cmip6/ESM4/DECK/ESM4_historical_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_cobalt_omip_2d/av/annual_5yr/'
        arch_dir_mon = '/archive/oar.gfdl.cmip6/ESM4/DECK/ESM4_historical_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_cobalt_omip_2d/av/monthly_5yr/'
        tim_inc = 5
        y1_store = 1995
    elif tim_per == "fut":
        arch_dir_ann = '/archive/oar.gfdl.bgrp-account/CMIP6/ESM4/ScenarioMIP/ESM4_ssp585_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_cobalt_omip_2d/av/annual_2yr/'
        arch_dir_mon = '/archive/oar.gfdl.bgrp-account/CMIP6/ESM4/ScenarioMIP/ESM4_ssp585_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_cobalt_omip_2d/av/monthly_2yr/'
        tim_inc = 2
        y1_store = 2081
    
    nlim_weighted_month  = np.zeros((12,nmod_lat,nmod_lon))
    plim_weighted_month  = np.zeros((12,nmod_lat,nmod_lon))
    felim_weighted_month = np.zeros((12,nmod_lat,nmod_lon))
    pp_month = np.zeros((12,nmod_lat,nmod_lon))
    
    for m in range(12):
        # create list of files
        y1 = y1_store
        nc_fils = []
        for fils in range(4):
            nc_fil = arch_dir_mon + 'ocean_cobalt_omip_2d.' + str(y1) + '-' + str(y1+tim_inc-1) + '.' + str(m+1).zfill(2) + '.nc'
            nc_fils.append(nc_fil)
            y1+=tim_inc
        fid_mod = nc.MFDataset(nc_fils)

        # Constituent productivity terms
        intppdiat = np.mean(fid_mod.variables['intppdiat'][:].squeeze(),axis=0)
        intppmisc = np.mean(fid_mod.variables['intppmisc'][:].squeeze(),axis=0)
        intpppico = np.mean(fid_mod.variables['intpppico'][:].squeeze(),axis=0)

        # Limitation terms (picophytoplankton)
        limnpico  = np.mean(fid_mod.variables['limnpico'][:].squeeze(),axis=0)
        limppico  = np.mean(fid_mod.variables['limppico'][:].squeeze(),axis=0)
        limfepico = np.mean(fid_mod.variables['limfepico'][:].squeeze(),axis=0)

        # Diatoms
        limndiat  = np.mean(fid_mod.variables['limndiat'][:].squeeze(),axis=0)
        limpdiat  = np.mean(fid_mod.variables['limpdiat'][:].squeeze(),axis=0)
        limfediat = np.mean(fid_mod.variables['limfediat'][:].squeeze(),axis=0)

        # Miscellaneous
        limnmisc  = np.mean(fid_mod.variables['limnmisc'][:].squeeze(),axis=0)
        limpmisc  = np.mean(fid_mod.variables['limpmisc'][:].squeeze(),axis=0)
        limfemisc = np.mean(fid_mod.variables['limfemisc'][:].squeeze(),axis=0)

        nlim_weighted_month[m,:]  = (intppdiat*limndiat + intppmisc*limnmisc + intpppico*limnpico)/\
                                    (intppdiat + intppmisc + intpppico)

        plim_weighted_month[m,:]  = (intppdiat*limpdiat + intppmisc*limpmisc + intpppico*limppico)/\
                                    (intppdiat + intppmisc + intpppico);

        felim_weighted_month[m,:] = (intppdiat*limfediat + intppmisc*limfemisc + intpppico*limfepico)/\
                                    (intppdiat + intppmisc + intpppico);

        pp_month[m,:] = (intppdiat + intpppico + intppmisc)*86400*num_days[m]*6.625*12 
    
    # Monthly weighted mean
    sum_pp_month = np.sum(pp_month,axis=0)

    nlim_weighted  = np.sum(nlim_weighted_month*pp_month,axis=0)/sum_pp_month
    plim_weighted  = np.sum(plim_weighted_month*pp_month,axis=0)/sum_pp_month
    felim_weighted = np.sum(felim_weighted_month*pp_month,axis=0)/sum_pp_month

    aa = ~np.isfinite(felim_weighted)

    felim = felim_weighted - np.min(np.stack((nlim_weighted,plim_weighted)),axis=0)
    felim[aa] = np.nan
    nlim  = nlim_weighted  - np.min(np.stack((felim_weighted,plim_weighted)),axis=0)
    nlim[aa] = np.nan
    plim  = plim_weighted  - np.min(np.stack((nlim_weighted,felim_weighted)),axis=0)
    plim[aa] = np.nan

    liebig_macro = np.min(np.stack((nlim_weighted,plim_weighted)),axis=0)
    liebig = np.min(np.stack((liebig_macro,felim_weighted)),axis=0)

    lim_ind = np.empty((nmod_lat,nmod_lon))
    lim_ind[nlim <= 0] = 1;
    lim_ind[(plim <= 0) * (plim > -0.25)] = 1.5
    lim_ind[plim <= -0.25] = 2
    lim_ind[(felim <= 0) * (felim > -0.25)] = 4.5
    lim_ind[felim <= -0.25] = 5
    
    return lim_ind


In [13]:
grid_file = '/archive/oar.gfdl.cmip6/ESM4/DECK/ESM4_historical_D1/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_annual/ocean_annual.static.nc';
fid_grd = nc.Dataset(grid_file)

lon_mod = fid_grd.variables['geolon'][:]
lat_mod = fid_grd.variables['geolat'][:]
area    = fid_grd.variables['areacello'][:]

nmod_lat, nmod_lon = lon_mod.shape

lat_crn = (lat_mod[1:,1:]+lat_mod[1:,:-1]+lat_mod[:-1,1:]+lat_mod[:-1,:-1])/4
lon_crn = (lon_mod[1:,1:]+lon_mod[1:,:-1]+lon_mod[:-1,1:]+lon_mod[:-1,:-1])/4

# Facilitate plotting tripolar grid on cartopy Mollweide projection
fixed_lons = lon_crn.copy()
for i, start in enumerate(np.argmax(np.abs(np.diff(lon_crn)) > 180, axis=1)):
    fixed_lons[i, start+1:] += 360
lon_crn = fixed_lons

num_days = [31,28,31,30,31,30,31,31,30,31,30,31] 

In [ ]:
tim_pers = ["his","fut"]
plt_tits = ['Historical (1995-2015) Nutrient Limitations',\
            'Future (2085-2100; ssp585) Nutrient Limitations']
legend_titles = ['N','Weakly P', 'P', 'Weakly Fe', 'Fe']

fig = plt.figure(figsize=(15,15))
spec = fig.add_gridspec(2, 1)

# Creating a Colormap
col_trm = ['blue', 'cornflowerblue','cyan', 'orangered', 'red']
mymap = plt_col.ListedColormap([col_trm[0], col_trm[1], col_trm[2], 'white', 'white', 'white', 'white', col_trm[3], col_trm[4]])
nrow = 0
for tim in tim_pers:
    plot_map(calc_lims(tim))
    nrow+=1

plt.savefig(('Past_and_Future_Nutrient_Limitations'))

In [6]:
#mod_file = arch_dir_ann + 'ocean_cobalt_omip_2d.1995-1999.ann.nc'
#fid_mod = nc.Dataset(mod_file)
#intpp_mod = 86400*1000*12*fid_mod.variables['intpp'][0,:].squeeze()
#aa = np.isfinite(intpp_mod)
#intpp_tot = np.dot(intpp_mod[aa],area[aa])/1e15

In [ ]:
#nc_fils = []
#for fils in range(4):
#    y1 = y1_store
#    nc_fil = arch_dir_ann + 'ocean_cobalt_omip_2d.' + str(y1) + '-' + str(y1+tim_inc-1) + '.ann.nc'
#    nc_fils.append(nc_fil)
#    y1+=tim_inc

#fid_mod = nc.MFDataset(nc_fils)
#intpn2 = 86400*1000*np.mean(fid_mod.variables['intpn2'][:].squeeze(),axis=0)
#aa = np.isfinite(intpn2)

# calculate the total primary production (Tg N yr-1)
#intpn2_tot = np.dot(intpn2[aa],area[aa])*365*14/(1000*1e12)